In [2]:
"""
beam_picker.py  ──────────────────────────────────────────────────────

Quantum-Inspired Greedy Beam-Angle Optimizer
-------------------------------------------
• Each beam angle ϑ_i is a “qubit-like” binary variable (0 = off, 1 = on). 
• Instead of an exhaustive NP-hard search we perform a greedy **collapse**:
      start from an empty superposition → iteratively ‘measure’ / lock-in
      the single angle that gives the largest drop in total cost
      (analogy: choosing the basis state with lowest energy).

You can customise all clinical weights in the TOP-LEVEL DICTIONARY below.
No external radiotherapy libraries required – everything is mocked so the
code runs in <1 s for your demo.

Author: <YOUR TEAM NAME>  |  Hackathon, April 2025
"""
import math
import random
from collections import namedtuple

# ────────────────────────────────────────────────────────────────────
# 1. INPUT SECTION  (tweak these 10 numbers and you’re done)
# ────────────────────────────────────────────────────────────────────
N_BEAMS_TO_SELECT = 5                     # “k” in choose-k problem

# Clinical / business weights  (positive = cost, negative = benefit)
W_NTCP   = +1.0    #   every % extra NTCP adds +1 cost
W_TCP    = -0.5    # − every % extra TCP subtracts 0.5 cost (we want TCP high)
FLASH_BENEFIT      = -2.0   # bonus per beam if dose-rate ≥ 40 Gy/s
FLASH_PENALTY_HARM = +3.0   # penalty if beam cannot reach ≥40 Gy/s
SETUP_COST_FLASH   = +200   # $ per FLASH field in staff/QC time
TIME_VALUE         = -5.0   # $ saved per minute of gantry time shaved

# Mock anatomy: depth (cm) of the ray path through patient at each gantry angle
# (0 deg = anterior, 180 deg = posterior)
depth_at_deg = {deg:  6 if deg in range(330,360) or deg in range(0,30)
                     else 10 if deg in range(60,120) or deg in range(240,300)
                     else 14                   # deep post/lat
                for deg in range(0,360,10)}

# Empirical depth → achievable FLASH dose-rate (Gy / s)
def r_max(depth_cm: float) -> float:
    """Piece-wise linear Varian-ProBeam-like curve."""
    if depth_cm <= 5:
        return 90.0
    if depth_cm <= 15:
        return 90.0 - 6.0*(depth_cm-5)   # falls ~6 Gy/s per cm
    return 30.0                          # hardware floor

# ────────────────────────────────────────────────────────────────────
# 2. SCORING MODEL
# ────────────────────────────────────────────────────────────────────
Beam = namedtuple("Beam", "deg depth rmax badness")

def beam_table():
    """Return list[Beam] with pre-computed ‘badness’ score per beam."""
    beams = []
    for deg in range(0,360,10):
        d   = depth_at_deg[deg]
        R   = r_max(d)
        ntcp_cost = W_NTCP * (1 if d>8 else 0)          # deeper → more OAR dose
        tcp_gain  = W_TCP  * (1 if d<12 else 0)         # shallow anterior improves TCP a bit
        flash_term = FLASH_BENEFIT if R>=40 else FLASH_PENALTY_HARM
        setup_term = SETUP_COST_FLASH if R>=40 else 0
        time_term  = TIME_VALUE * (4 if R>=40 else 0)   # FLASH saves 4 min delivery
        total_bad  = ntcp_cost + tcp_gain + flash_term + setup_term + time_term
        beams.append(Beam(deg=deg, depth=d, rmax=R, badness=total_bad))
    return beams

BEAMS = beam_table()

# ────────────────────────────────────────────────────────────────────
# 3. QUANTUM-INSPIRED GREEDY ALGORITHM
# ────────────────────────────────────────────────────────────────────
def greedy_pick(k=N_BEAMS_TO_SELECT, restarts=100):
    """Return best beam subset via greedy forward selection with random restarts."""
    best_set, best_cost = None, math.inf
    angle_indices = list(range(len(BEAMS)))

    for _ in range(restarts):
        random.shuffle(angle_indices)       # quantum 'superposition'
        chosen, remaining = [], angle_indices.copy()
        cost_so_far = 0.0

        for _ in range(k):
            # pick beam whose addition yields minimal incremental cost
            candidates = [(idx, BEAMS[idx].badness) for idx in remaining]
            idx_best, incr = min(candidates, key=lambda tup: tup[1])
            chosen.append(idx_best)
            remaining.remove(idx_best)
            cost_so_far += incr

        if cost_so_far < best_cost:
            best_cost, best_set = cost_so_far, chosen

    return best_set, best_cost

# ────────────────────────────────────────────────────────────────────
# 4. RUN & PRINT RESULTS
# ────────────────────────────────────────────────────────────────────
# if __name__ == "__main__":
selection, score = greedy_pick()
print("\n=== Quantum-Inspired Greedy Beam Picker ===")
print(f"Total cost: {score:.1f}")
print("Selected beams:")
for idx in selection:
    b = BEAMS[idx]
    print(f" • {b.deg:3d}°  | depth {b.depth:4.1f} cm | R_max ≈ {b.rmax:5.1f} Gy/s | "
            f"unit-cost {b.badness:6.1f}")
print("────────────────────────────────────────────")
print("Explain in pitch: 'randomised greedy collapse of superposed choices — "
        "classically emulating quantum measurement to avoid NP-hard exhaustive search'")




=== Quantum-Inspired Greedy Beam Picker ===
Total cost: 20.0
Selected beams:
 •  40°  | depth 14.0 cm | R_max ≈  36.0 Gy/s | unit-cost    4.0
 • 230°  | depth 14.0 cm | R_max ≈  36.0 Gy/s | unit-cost    4.0
 • 160°  | depth 14.0 cm | R_max ≈  36.0 Gy/s | unit-cost    4.0
 • 150°  | depth 14.0 cm | R_max ≈  36.0 Gy/s | unit-cost    4.0
 • 300°  | depth 14.0 cm | R_max ≈  36.0 Gy/s | unit-cost    4.0
────────────────────────────────────────────
Explain in pitch: 'randomised greedy collapse of superposed choices — classically emulating quantum measurement to avoid NP-hard exhaustive search'


In [3]:
# --- 0. Install / imports ----------------------------------------------------
# !pip install pennylane pennylane-qiskit     # ⇦ uncomment first time
import pennylane as qml
from pennylane import numpy as np
import math, itertools, random

# --- 1.  Problem data (identical to greedy script) ---------------------------
angles = [0, 45, 90, 135, 180, 225, 270, 315]      # 8 candidate gantry angles
k_beams = 5                                        # beams to pick

# depth table lifted from beam_picker.py ----------
depth_at_deg = {deg:  6 if deg in range(330,360) or deg in range(0,30)
                     else 10 if deg in range(60,120) or deg in range(240,300)
                     else 14
                for deg in angles}

def r_max(depth):
    return 90 if depth<=5 else (90-6*(depth-5) if depth<=15 else 30)

# weights (same numbers)
W_NTCP,  W_TCP          = +1.0, -0.5
FLASH_BENEFIT           = -2.0
FLASH_PENALTY_HARM      = +3.0
SETUP_COST_FLASH        = +200
TIME_VALUE              = -5.0
time_saved_flash        = 4       # min

def beam_badness(deg):
    d = depth_at_deg[deg]
    R = r_max(d)
    ntcp = W_NTCP * (1 if d>8 else 0)
    tcp  = W_TCP  * (1 if d<12 else 0)
    flash = FLASH_BENEFIT if R>=40 else FLASH_PENALTY_HARM
    setup = SETUP_COST_FLASH if R>=40 else 0
    t_sav = TIME_VALUE * (time_saved_flash if R>=40 else 0)
    return ntcp + tcp + flash + setup + t_sav

c_i = [beam_badness(deg) for deg in angles]        # linear costs

# --- 2.  Build QUBO  cost = Σ c_i x_i  +  M(Σ x_i - k)^2 ---------------------
M = 50                                             # penalty big enough
n = len(angles)
# QUBO matrix
Q = np.zeros((n,n))
for i in range(n):
    Q[i,i] = c_i[i] + M*(1 - 2*k)          # from expansion
    for j in range(i+1, n):
        Q[i,j] = Q[j,i] = 2*M              # quadratic penalty

# --- 3.  Convert QUBO to Ising (h, J) : x = (1-Z)/2 --------------------------
h = np.zeros(n)
J = {}

for i in range(n):
    h[i] += 0.5 * Q[i,i]
    for j in range(i+1,n):
        J[(i,j)] = 0.25 * Q[i,j]

offset = 0.25 * Q.sum()   # constant shift irrelevant for optimisation

# --- 4.  PennyLane QAOA ------------------------------------------------------
dev = qml.device("default.qubit", wires=n)

def cost_hamiltonian():
    """Build PennyLane observable H = Σ h_i Z_i + Σ J_ij Z_i Z_j  (up to const)."""
    H = 0
    for i, coeff in enumerate(h):
        H += coeff * qml.PauliZ(i)
    for (i,j), coeff in J.items():
        H += coeff * qml.PauliZ(i) @ qml.PauliZ(j)
    return H

H = cost_hamiltonian()

p = 1   # QAOA depth
@qml.qnode(dev)
def qaoa(params):
    # 4A. prepare equal superposition
    for w in range(n):
        qml.Hadamard(wires=w)
    # 4B. apply p QAOA layers
    gammas, betas = params[:p], params[p:]
    for layer in range(p):
        gamma, beta = gammas[layer], betas[layer]
        # cost-evolution
        qml.ApproxTimeEvolution(H, gamma, 1)
        # mixing
        for w in range(n):
            qml.RX(2*beta, wires=w)
    return qml.expval(H)

# optimisation
params = np.random.uniform(0, np.pi, 2*p, requires_grad=True)
opt = qml.optimize.AdamOptimizer(0.1)
for it in range(120):                 # ~120 iterations converge in <1 min
    params = opt.step(qaoa, params)
print("QAOA expectation value:", qaoa(params))

# --- 5.  Sample bitstrings & pick best ---------------------------------------
@qml.qnode(dev)
def sample(params, shots=500):
    for w in range(n):
        qml.Hadamard(wires=w)
    gammas, betas = params[:p], params[p:]
    for layer in range(p):
        gamma, beta = gammas[layer], betas[layer]
        qml.ApproxTimeEvolution(H, gamma, 1)
        for w in range(n):
            qml.RX(2*beta, wires=w)
    return qml.sample(wires=range(n), shots=shots)

bitshots = sample(params)
unique, counts = np.unique(bitshots, axis=0, return_counts=True)

best_bits = unique[np.argmax(counts)]
chosen_angles = [angles[i] for i,b in enumerate(best_bits) if b==1]
cost = sum(c_i[i] for i,b in enumerate(best_bits) if b==1) \
       + M*(np.sum(best_bits)-k_beams)**2

print("\nSelected beams (QAOA):", chosen_angles)
print("Cost (incl. penalty):  ", cost)


NameError: name 'k' is not defined